# Backing PQL Queries with Data - thoughts on implementation

The following section describe how to map and implement each clause of PQL queries to corresponding data
if queries.

I decided to implement the data backend in pandas as well. This is in order to reduce the work stack complexity. The alternative would be to implement it using some ORM (Object-Relational-Mapper) (e.g. SQL-Alchemy) and some SQL DBMS (e.g. sqlite3). But it appears to me that this add drastic new complexity and requries me to learn a lot of new things that I can implement more easilty using pandas. 

## Handling of Data - or: when to copy

naively we could just copy data on each copy of a model. that works. However, we could also copy data whenever we change it instead, but when copying a whole model just copying a reference (as it is unchanged). Also, we can leave it unchanged when we only require a few on (parts of) the data. This could help save quite some space and time. I'm unsure about the implications of object serialization, however.

Also, this required probably good knowledge about the view-vs-copy issue: read about it!
http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

## Order of 'excecution' of clauses

it appears to be entirely analogous to the model 'predict' workflow:

1. get sub data-frame of interest
2. apply non-aggregation/density filters
3. do splitting
4. calculate aggregations and frequencies
2. apply aggregation/frequency filters

## SPLIT-clause 

Group-By (i.e. the .groupby method on Pandas DataFrames) generates group by mean of some mapping of labels to group names. Such a mapping can be anything, and most importantly also a function which returns a group name for each label.

In [8]:
# load some data frame to play with
import pandas as pd
import numpy as np
df = pd.read_csv('../data/crabs/australian-crabs.csv')
print(df.head())

  species   sex  index   FL   RW    CL    CW   BD
0    Blue  Male      1  8.1  6.7  16.1  19.0  7.0
1    Blue  Male      2  8.8  7.7  18.1  20.8  7.4
2    Blue  Male      3  9.2  7.8  19.0  22.4  7.7
3    Blue  Male      4  9.6  7.9  20.1  23.1  8.2
4    Blue  Male      5  9.8  8.0  20.3  23.0  8.2


Splitting by elements is easy: just use groupby(columnname)

In [6]:
columns = ['sex','species']
group = df.groupby(columns)
print('keys: ', group.keys)
print('groups: ', group.groups)


keys:  ['sex', 'species']
groups:  {('Male', 'Blue'): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], ('Female', 'Blue'): [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], ('Male', 'Orange'): [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149], ('Female', 'Orange'): [150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 19

But how about splitting into bins? i.e. split that use the equiDist method in my PQL queries?
I think the way to go is to first do binning and add a group column (index) as a result of that binning. That index can then be used together with those column that we 'elements'-split on.

In [11]:
colname = 'RW'  # column to bin 
bin_edges = np.arange(11)  # sequence of bin edges. in real this is gonna be 
bin_colname = 'bins'  # need to use some generator for unique names later
 
df[bin_colname] = pd.cut(df[colname], bin_edges, include_lowest=True)  # include lower edge to cover the entire range

elem_split_names = ['sex']  # other columns to split by by elements

# now do entire grouping at once
grps = df.groupby(elem_split_names + [bin_colname])

In [12]:
print(grps.groups)

{('Male', '(9, 10]'): [6, 7, 8, 12, 104, 105], ('Female', nan): [62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199], ('Male', '(8, 9]'): [5, 101, 102, 103], ('Female', '(8, 9]'): [51, 52, 53, 54, 55], ('Female', '(9, 10]'): [56, 57, 58, 59, 60, 61, 150, 151, 152], ('Male', '(6, 7]'): [0, 100], ('Male', nan): [9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,

## PREDICT-clause

### PREDICT-clause (no aggregation)

that's easy: just select the columns using some of the many indexing methods

In [13]:
colnames = ['FL','sex']

using .loc

In [17]:
selection = df.loc[:,colnames]
print(selection.head())

    FL   sex
0  8.1  Male
1  8.8  Male
2  9.2  Male
3  9.6  Male
4  9.8  Male


using the [] operator:

In [19]:
selection = df[colnames]
print(selection.head())

    FL   sex
0  8.1  Male
1  8.8  Male
2  9.2  Male
3  9.6  Male
4  9.8  Male


### PREDICT-clause (aggregations)

we want to support the following aggregations:
  
  * average: average value of a group 
  * maximum: maximum value within a group
  

### PREDICT-clause (density)

uh... not sure yet. we could of course return the size of each group, i.e. the frequency of observaion

## WHERE-clause

we have a number of difference condition/filter/where's that we want to support:

  * equals (a singluar value)
  * in (element of a set of values)
  * in (a range of [min, max]
  * greater 
  * less

Generally it seems like we can express everything we want using .loc and selecitons by callables.
The point of callables is that we can chain, since it is evaluated on the object it is called on rather than some local variable  

### WHERE-clause (equals)

In [20]:
value = 'Female'
colname = 'sex'
filtered = df.loc[df[colname] == value]
print(filtered.head())
#or
filtered = df.loc[lambda df:df[colname] == value]
print(filtered.head())

   species     sex  index   FL   RW    CL    CW   BD    bins
50    Blue  Female      1  7.2  6.5  14.7  17.1  6.1  (6, 7]
51    Blue  Female      2  9.0  8.5  19.3  22.7  7.7  (8, 9]
52    Blue  Female      3  9.1  8.1  18.5  21.6  7.7  (8, 9]
53    Blue  Female      4  9.1  8.2  19.2  22.2  7.7  (8, 9]
54    Blue  Female      5  9.5  8.2  19.6  22.4  7.8  (8, 9]
   species     sex  index   FL   RW    CL    CW   BD    bins
50    Blue  Female      1  7.2  6.5  14.7  17.1  6.1  (6, 7]
51    Blue  Female      2  9.0  8.5  19.3  22.7  7.7  (8, 9]
52    Blue  Female      3  9.1  8.1  18.5  21.6  7.7  (8, 9]
53    Blue  Female      4  9.1  8.2  19.2  22.2  7.7  (8, 9]
54    Blue  Female      5  9.5  8.2  19.6  22.4  7.8  (8, 9]


### WHERE-clause (in set of elements)

In [23]:
values = ['Female']
col = 'sex'
filtered = df.loc[df[col].isin(values)]
print(filtered.head())
#or
filtered = df.loc[lambda df: df[col].isin(values)]
print(filtered.head())

   species     sex  index   FL   RW    CL    CW   BD    bins
50    Blue  Female      1  7.2  6.5  14.7  17.1  6.1  (6, 7]
51    Blue  Female      2  9.0  8.5  19.3  22.7  7.7  (8, 9]
52    Blue  Female      3  9.1  8.1  18.5  21.6  7.7  (8, 9]
53    Blue  Female      4  9.1  8.2  19.2  22.2  7.7  (8, 9]
54    Blue  Female      5  9.5  8.2  19.6  22.4  7.8  (8, 9]
   species     sex  index   FL   RW    CL    CW   BD    bins
50    Blue  Female      1  7.2  6.5  14.7  17.1  6.1  (6, 7]
51    Blue  Female      2  9.0  8.5  19.3  22.7  7.7  (8, 9]
52    Blue  Female      3  9.1  8.1  18.5  21.6  7.7  (8, 9]
53    Blue  Female      4  9.1  8.2  19.2  22.2  7.7  (8, 9]
54    Blue  Female      5  9.5  8.2  19.6  22.4  7.8  (8, 9]


### WHERE-clause (in a range)

this is a combination of 'less' and 'greater' ... anyway:

In [24]:
r = {'l': 8.1, 'h': 10}
col = 'FL'
filtered = df.loc[lambda df:(df.FL < r['h']) & (df.FL > r['l'])]
print(filtered.head())

   species     sex  index   FL   RW    CL    CW   BD    bins
1     Blue    Male      2  8.8  7.7  18.1  20.8  7.4  (7, 8]
2     Blue    Male      3  9.2  7.8  19.0  22.4  7.7  (7, 8]
3     Blue    Male      4  9.6  7.9  20.1  23.1  8.2  (7, 8]
4     Blue    Male      5  9.8  8.0  20.3  23.0  8.2  (7, 8]
51    Blue  Female      2  9.0  8.5  19.3  22.7  7.7  (8, 9]


### WHERE-clause (less and greater)

this is just like 'in a range' but simpler ...